# Lesson 3 : Deploying a token sales contract


## Writing a smart contract that allows sales of our token from last week.

We will be using the Crowdsales Open Zeppelin contract as our base contract
https://docs.openzeppelin.com/contracts/2.x/crowdsales

## Deploying to testnet 
## Writing python code to interact with our new sales contract


### Load all the python libraries that we need to use

In [ ]:
from web3 import Web3
from datetime import datetime

import urllib
import json
import pandas as pd

### Load our API and pvt keys 


In [ ]:
with open("..\..\.env", "r") as f:
    keys = f.read().splitlines()
    
for key in keys:
    if "ETHERSCAN_API_KEY" in key: etherscan_apikey = key.split('=')[1].strip('\"')
    if "ALCHEMY_KEY" in key: alchemy_apikey = key.split('=')[1].strip('\"')
    if "ACCOUNT_PRIVATE_KEY" in key: pvt_key = key.split('=')[1].strip('\"')
    

### Connect to the Alchemy provider
To interact with the Ethereum blockchain, we either need to run a geth node on our machine, or we can use a third party provider like Alchemy or Infura

In [ ]:
wss = f'wss://eth-goerli.g.alchemy.com/v2/{alchemy_apikey}'
wss = f'wss://eth-sepolia.g.alchemy.com/v2/{alchemy_apikey}'
w3 = Web3(Web3.WebsocketProvider(wss))

print(w3.is_connected())

In [ ]:
#We should be connected to chain_id 5, i.e. Goerli testnet, or 11155111 for Sepolia
w3.eth.chain_id

### Token contract addresses 

Normally we would like to use the proxy pattern, but in this case we are using Open Zeppelin's Crowdsale contract which is only available in non-upgradeable form. We could re-write it as an upgradeable in the future

In [ ]:
# TokenSale Contract Address
tokensale_contract = '0x856820ee9EE8137D1c651689Ab6D548f5ac81fA1'

# whipz contracts
# Implementation Address
whipz_implementation_contract = '0x7006fae1c7746b3aee4baabb3941875984090252'

whipz_proxy = '0x9f154d7b7a00a2a280733409aea309d2aea496f8'

### Load the ABI (Application Binary Interface) for both contracts

The ABI defines all the functions and function parameters of the smart contract. We need this info if we want to make calls to the contract

In [ ]:
def get_contract(impl_contract, proxy_contract):

    abi_endpoint = f"https://api-sepolia.etherscan.io/api?module=contract&action=getabi&address={impl_contract}&apikey={etherscan_apikey}"
    
    # Get the abi in json format from the etherscan API and decode it
    with urllib.request.urlopen(abi_endpoint) as url:
        abi = json.loads(url.read().decode())

    print(abi)
    # Load this into the web3 contract object
    checksum_addr = Web3.to_checksum_address(proxy_contract)
    contract = w3.eth.contract(address=checksum_addr, abi=abi["result"])
    return contract


In [ ]:
whipzcontract = get_contract(whipz_implementation_contract, whipz_proxy)

In [ ]:
tokensales_contract = get_contract(tokensale_contract, tokensale_contract)

In [ ]:
tokensales_contract.functions.rate().call()

In [ ]:
whipzcontract.functions.totalSupply().call()

In [ ]:
whipzcontract.functions.name().call()

In [ ]:
whipzcontract.functions. symbol().call()

# Give the TokenSale contract approval to spend Whipz

In [ ]:
MAX_INT = 2**256 - 1
admin_wallet = '0xa90B01e34D5eB0eF48ece9F23c4c953568440147'
nonce = w3.eth.get_transaction_count(admin_wallet)
GAS_AMOUNT= 100000

In [ ]:
call_function = whipzcontract.functions.approve(
    tokensale_contract, MAX_INT).build_transaction(
    {"chainId": w3.eth.chain_id,
     "from": admin_wallet,
     "nonce": nonce,
     'gas': GAS_AMOUNT,
#      'value': total_amt,
     'maxFeePerGas': w3.to_wei(100, 'gwei'),
     'maxPriorityFeePerGas': w3.to_wei(1, 'gwei'), })

In [ ]:
signed_tx = w3.eth.account.sign_transaction(call_function, private_key=pvt_key)

In [ ]:
send_tx = w3.eth.send_raw_transaction(signed_tx.rawTransaction)

In [ ]:
send_tx

# Mint Whipz to TokenSale address

In [ ]:
mint_amount =  int(10e18 * 10e9)
nonce = w3.eth.get_transaction_count(admin_wallet)

In [ ]:
mint_amount

In [ ]:
call_function2 = whipzcontract.functions.mint(
    tokensale_contract, mint_amount).build_transaction(
    {"chainId": w3.eth.chain_id,
     "from": admin_wallet,
     "nonce": nonce,
     'gas': GAS_AMOUNT,
#      'value': total_amt,
     'maxFeePerGas': w3.to_wei(100, 'gwei'),
     'maxPriorityFeePerGas': w3.to_wei(1, 'gwei'), })

In [ ]:
w3.eth.estimate_gas(call_function2)

In [ ]:
signed_tx = w3.eth.account.sign_transaction(call_function2, private_key=pvt_key)

In [ ]:
send_tx = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
send_tx

# Purchase Whipz from the TokenSale 

In [ ]:
purchase_amt = 0.01 #this is the amount of ETH we are buying with

nonce = w3.eth.get_transaction_count(admin_wallet)

In [ ]:
call_function3 = tokensales_contract.functions.buyTokens(admin_wallet).build_transaction(
    {"chainId": w3.eth.chain_id,
     "from": admin_wallet,
     "nonce": nonce,
     'gas': GAS_AMOUNT,
     'value': w3.to_wei(purchase_amt, 'ether'),
     'maxFeePerGas': w3.to_wei(100, 'gwei'),
     'maxPriorityFeePerGas': w3.to_wei(1, 'gwei'), })

In [ ]:
w3.eth.estimate_gas(call_function3)

In [ ]:
signed_tx = w3.eth.account.sign_transaction(call_function3, private_key=pvt_key)

In [ ]:
send_tx = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
send_tx